# Import librerie e definizione funzioni d'utilità

In [ ]:
import functions as fn

In [ ]:
import pandas as pd
import numpy as np
import os
import geopandas as gpd

In [ ]:
dataset_name = "infoclipper"
if not os.path.exists("..\\Plot"):
    os.mkdir("..\\Plot")
if not os.path.exists("..\\Plot\\" + dataset_name):
    os.mkdir("..\\Plot\\" + dataset_name)
if not os.path.exists("..\\Plot\\" + dataset_name + "\\tempi"):
    os.mkdir("..\\Plot\\" + dataset_name + "\\tempi")
if not os.path.exists("..\\DatasetParsed"):
    os.mkdir("..\\DatasetParsed")
if not os.path.exists("..\\DatasetParsed\\" + dataset_name):
    os.mkdir("..\\DatasetParsed\\" + dataset_name)

In [ ]:
counter = 0
def make_path(dataset, file_name, other = ""):
    global counter
    counter += 1
    return "..\\Plot\\" + dataset + "\\" + other + str(counter) + "-" + file_name

# Estrazione dati tabella aziende

In [ ]:
dataset_ic = "..\\Datasets\\" + dataset_name + "\\info-clipper-ds.csv"
df_ic = pd.read_csv(dataset_ic, encoding='cp1252')
df_time = df_ic[["URL", "Request time", "tempo Estrazione Name", "tempo estrazione Trade Name", "tempo estrazione Address", "tempo estrazione City", "tempo estrazione State", "tempo estrazione Country", "tempo estrazione Location Type"]]
df_ic = df_ic[["URL", "ID", "Name", "Trade Name", "Address Name", "Postalcode", "City", "State", "Country", "Location type"]]
print("Numero entry tabella originale: ", len(df_ic))
df_ic = df_ic.dropna(subset=["URL"], axis=0)
df_ic['ID'] = df_ic['ID'].astype('int')
print("Filtro URL null: ", len(df_ic))

In [ ]:
df_ic.to_csv("..\\DatasetParsed\\" + dataset_name + "\\" + dataset_name + "-aziende.csv", index=False)
df_ic.head()

In [ ]:
info_dataset = fn.extract_data_from_df(df_ic)
display(pd.DataFrame(info_dataset, index=["valore"]).T)

### Statistiche celle

In [ ]:
keys = ["celle totali", "celle nulle"]
values = [
    info_dataset["numero celle"], 
    info_dataset["numero celle nulle"]
]
title = "Confronto numero celle totali e vuote"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (7,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name), dim)

In [ ]:
keys = ["celle totali", "celle str", "celle float", "celle int"]
values = [
    info_dataset["numero celle"], 
    info_dataset["numero celle str"],
    info_dataset["numero celle float"],
    info_dataset["numero celle int"]
]
title = "Confronto tipo celle"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (7,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name), dim)

### Statistiche righe

In [ ]:
keys = ["righe totali", "righe con celle nulle"]
values = [
    info_dataset["numero righe"], 
    info_dataset["numero righe con celle nulle"]
]
title = "Confronto numero righe totali e nulle"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (7,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name), dim)

### Statistiche colonne

In [ ]:
keys = ["colonne totali", "colonne con celle nulle"]
values = [
    info_dataset["numero colonne"], 
    info_dataset["numero colonne con celle nulle"]
]
title = "Confronto numero colonne totali e nulle"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (7,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name), dim)

In [ ]:
keys = ["colonne totali", "colonne str", "colonne float", "colonne int"]
values = [
    info_dataset["numero colonne"], 
    info_dataset["numero colonne str"],
    info_dataset["numero colonne float"],
    info_dataset["numero colonne int"]
]
title = "Confronto tipo colonne"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (7,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name), dim)

In [ ]:
different_value_columns = {}
for key in df_ic:
    different_value_columns[key] = [len(pd.unique(df_ic[key])), df_ic[key].isnull().sum()]
different_value_columns = dict(sorted(different_value_columns.items(), key=lambda item: item[1], reverse=True))

In [ ]:
keys = different_value_columns.keys()
values = [item[0] for item in different_value_columns.values()]
title = "Numero di valori distinti per colonna"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (8,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name), dim, True, info_dataset["numero righe"])

In [ ]:
keys = different_value_columns.keys()
values = [item[1] for item in different_value_columns.values()]
title = "Numero di valori nulli per colonna"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (8,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name), dim, True, info_dataset["numero righe"])

### Plot su mappe geografiche

In [ ]:
df_us = df_ic[df_ic["Country"] == "UNITED STATES"]
df_uk = df_ic[df_ic["Country"] == "UNITED KINGDOM"]
df_it = df_ic[df_ic["Country"] == "ITALY"]
df_es = df_ic[df_ic["Country"] == "ESTONIA"]

##### Stati Uniti d'America

In [ ]:
def clear_state(x):
    return x.split("(")[0]
df_us["State"] = df_us["State"].apply(clear_state)

In [ ]:
dict_of_nation = {}
nation = list(df_us["State"])
for n in nation:
    dict_of_nation[n] = nation.count(n)

In [ ]:
fp = "..\\Mappe\\" + dataset_name + "\\USA\\usa.shp"
map_df = gpd.read_file(fp)
map_df["n_companies"] = np.nan
for i, row in map_df.iterrows():
    if row[5] in dict_of_nation:
        map_df.at[i, 'n_companies'] = dict_of_nation[row[5]]
    else:
        map_df.at[i, 'n_companies'] = 0
map_df = map_df[map_df.NAME != "Alaska"]
map_df = map_df[map_df.NAME != "Hawaii"]
map_df = map_df[map_df.NAME != "Puerto Rico"]
map_df = map_df[map_df.NAME != "American Samoa"]
map_df = map_df[map_df.NAME != "United States Virgin Islands"]
map_df = map_df[map_df.NAME != "Guam"]
map_df = map_df[map_df.NAME != "Commonwealth of the Northern Mariana Islands"]
map_df.head()

In [ ]:
fn.map_plot(map_df, "n_companies", "STUSPS", "geometry", 10,"Numero di aziende per stato", make_path(dataset_name, "usa"))

##### Inghilterra

In [ ]:
file_postcodes = "..\\CAP\\" + dataset_name + "\\postcodes.csv"
postcodes = pd.read_csv(file_postcodes)
postcodes = postcodes[['Postcode', 'UK region']]
postcodes = postcodes.rename(columns={"Postcode": "postcode", "UK region": "region"})

In [ ]:
caps = list(df_uk["Postalcode"])
region = []
for cap in caps:
    entry = postcodes.loc[postcodes.postcode.apply(lambda x: cap.strip().startswith(x))]
    if len(entry.region.values) > 0:
        region.append(entry.region.values[0])

In [ ]:
n_companies_by_region = {}
res = []
[res.append(x) for x in region if x not in res]
for c in res:
    n_companies_by_region[c] = region.count(c)

In [ ]:
fp = "..\\Mappe\\" + dataset_name + "\\Inghilterra\\inghilterra.shp"
map_df = gpd.read_file(fp)
map_df["n_companies"] = np.nan
for i, row in map_df.iterrows():
    if row[2] in n_companies_by_region:
        map_df.at[i, 'n_companies'] = n_companies_by_region[row[2]]
map_df.head()

In [ ]:
fn.map_plot(map_df, "n_companies", "rgn17nm", "geometry", 10,"Numero di aziende per regione", make_path(dataset_name, "inghilterra"))

##### Italia

In [ ]:
file_postcodes = "..\\CAP\\" + dataset_name + "\\codici_postali_ITALIA.csv"
postcodes = pd.read_csv(file_postcodes, encoding='cp1252')

In [ ]:
caps = list(df_it["Postalcode"])
region = []
for cap in caps:
    entry1 = postcodes.loc[postcodes.CAP1.apply(lambda x: x <= int(cap))]
    entry2 = postcodes.loc[postcodes.CAP2.apply(lambda x: x >= int(cap))]
    int_df = pd.merge(entry1, entry2, how ='inner')
    if len(int_df.Regione.values) > 0:
        region.append(int_df.Regione.values[0])

In [ ]:
n_companies_by_region = {}
res = []
[res.append(x) for x in region if x not in res]
for c in res:
    n_companies_by_region[c] = region.count(c)

In [ ]:
fp = "..\\Mappe\\" + dataset_name + "\\Italia\\italia.shp"
map_df = gpd.read_file(fp)
map_df.at[1, "NAME_1"] = "Puglia"
map_df.at[14, "NAME_1"] = "Sicilia"
map_df["n_companies"] = np.nan
for i, row in map_df.iterrows():
    if row[4] in n_companies_by_region:
        map_df.at[i, 'n_companies'] = n_companies_by_region[row[4]]
map_df.head()

In [ ]:
fn.map_plot(map_df, "n_companies", "NAME_1", "geometry", 10,"Numero di aziende per regione", make_path(dataset_name, "nazioniUK"))

##### Estonia

In [ ]:
file_postcodes = "..\\CAP\\" + dataset_name + "\\codici_postali_ESTONIA.csv"
postcodes = pd.read_csv(file_postcodes, encoding='cp1252')

In [ ]:
caps = [x.strip() for x in list(df_es["Postalcode"]) if x != " "]
region = []
for cap in caps:
    entry1 = postcodes.loc[postcodes.CAP1.apply(lambda x: x <= int(cap))]
    entry2 = postcodes.loc[postcodes.CAP2.apply(lambda x: x >= int(cap))]
    int_df = pd.merge(entry1, entry2, how ='inner')
    if len(int_df.Regione.values) > 0:
        region.append(int_df.Regione.values[0])

In [ ]:
n_companies_by_region = {}
res = []
[res.append(x) for x in region if x not in res]
for c in res:
    n_companies_by_region[c] = region.count(c)

In [ ]:
fp = "..\\Mappe\\" + dataset_name + "\\Estonia\\estonia.shp"
map_df = gpd.read_file(fp)
map_df["n_companies"] = np.nan
map_df["MNIMI"] = map_df["MNIMI"].apply(lambda x: x.split(" ")[0])
for i, row in map_df.iterrows():
    if row[0] in n_companies_by_region:
        map_df.at[i, 'n_companies'] = n_companies_by_region[row[0]]
map_df.head()

In [ ]:
fn.map_plot(map_df, "n_companies", "MNIMI", "geometry", 10,"Numero di aziende per contea", make_path(dataset_name, "contee_estonia"))

# Estrazione prestazioni

In [ ]:
# Tempo totale di esecuzione 1: 231.700044631958 (secondi), 420 entry
# Tempo totale di esecuzione 2: 231.72814297676086 (secondi), 440 entry
# Tempo totale di esecuzione 4: 250.68748903274536 (secondi), 420 entry

In [ ]:
df_time.head()

In [ ]:
new_df_time = df_time.describe()
del new_df_time["Request time"]

In [ ]:
keys = ["Tempo richiesta", "Tempo estrazione dettagli"]
values = [df_time.describe()["Request time"]["mean"], new_df_time.sum(axis="columns")["mean"]]
values = np.array(values) * 1000
title = "Confronto tempo medio (ms) request vs tempo medio (ms) estrazione dettagli"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (7,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name, "tempi\\"), dim, True)

In [ ]:
keys = list(new_df_time.loc["mean"].keys())
values = list(new_df_time.loc["mean"])
values = np.array(values) * 1000000
title = "Tempi medi (\u03BCs) di estrazione dei valori (solo xpath)"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (10,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name, "tempi\\"), dim, True)

In [ ]:
keys = list(new_df_time.loc["max"].keys())
values = list(new_df_time.loc["max"])
values = np.array(values) * 1000
title = "Tempo massimo (ms) di estrazione dei valori (solo xpath)"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (10,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name, "tempi\\"), dim, True)